In [51]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import requests
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
import regex as re
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.neighbors import  KNeighborsClassifier 
from sklearn.metrics import r2_score,mean_squared_error,confusion_matrix,accuracy_score
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

import ast

We are scrapping data from yelp and saving it into a csv file. We scrapped 5000 values first then the north, east, south, and west parts. This was followed by specific areas that were not covered. Code was provided by Yelp.

In [23]:
URL = 'https://api.yelp.com/v3/businesses/search?location=New York City'
API_key = 'Wt1iPgJtxRzatFu8BvZnmPf6mGueXES6NtxqXWoRe3eyBq90wsmxW7D43ye2RQRMJ72Q-gaYbXoOgBBB4kn0WX4b87Tk2U1YKtgVjvLd_WvmGRoF0eLAw_bhtsFCXnYx'
res = requests.get(URL)
res.status_code

400

In [108]:
# Scrap Yelp data
cities = ['new york']
data = []
count = 1
for city in cities:
    for i in range(20):
        URL = 'https://api.yelp.com/v3/businesses/search?location=New York City'
        API_key = 'Wt1iPgJtxRzatFu8BvZnmPf6mGueXES6NtxqXWoRe3eyBq90wsmxW7D43ye2RQRMJ72Q-gaYbXoOgBBB4kn0WX4b87Tk2U1YKtgVjvLd_WvmGRoF0eLAw_bhtsFCXnYx'
        params = {'limit': 50 ,
                  'offset': 50 * i}
        headers = {'Authorization': 'bearer %s' % API_key}
        resp = requests.get(url=URL, params = params, headers = headers)
        data.extend(resp.json()['businesses'])
        if len(data) == 0:
            break
        print(f'Pulling {count} times.')
        count += 1
        time.sleep(.5)

In [4]:
# This is done three times with the files name yelp1, yelp2, and yelp 3
df = pd.DataFrame(data = data)
df.to_csv('yelp1.csv', index = False)

In [80]:
yelp1 = pd.read_csv('datasets/yelp1.csv')

In [1]:
# Scrap Yelp data
def query_pushshift(location, kind = 'submission'):
    SUBFIELDS = ['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'categories', 'rating', 'coordinates', 'transactions', 'price', 'location', 'phone', 'display_phone', 'distance']
#define list up here to store infromation ebing done down there
    business = []
    for i in range(20):
        url = 'https://api.yelp.com/v3/businesses/search'
        API_key = 'Wt1iPgJtxRzatFu8BvZnmPf6mGueXES6NtxqXWoRe3eyBq90wsmxW7D43ye2RQRMJ72Q-gaYbXoOgBBB4kn0WX4b87Tk2U1YKtgVjvLd_WvmGRoF0eLAw_bhtsFCXnYx'
        headers = { "Authorization": f'Bearer {API_key}'}
        api_params = {'location' : {location},
              'limit' : 50,
              'offset': i* 50}
        print("Querying from: " + url)
        res = requests.get(url, headers = headers, params= api_params )
        assert res.status_code == 200
        mine = res.json()['businesses']
        df = pd.DataFrame.from_dict(mine)
        business.append(df)
        time.sleep(3)
        full = pd.concat(business, sort=False)
        if kind == "submission":
        # select desired columns
            full = full[SUBFIELDS]
    print("Query Complete!")    
    return full 

In [6]:
brooklyn_heights = query_pushshift('76 Montague St, Brooklyn, NY 11201')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [8]:
williamsburg = query_pushshift('533 Grand St, Brooklyn, NY 11211')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [9]:
long_island_city = query_pushshift('2103 45th Ave, Long Island City, NY 11101')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [10]:
astoria = query_pushshift('32-24 30th Ave, Astoria, NY 11102')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [18]:
df =pd.concat([brooklyn_heights,williamsburg,astoria,long_island_city])

In [19]:
locations = df['location']
zip_codes = [ sub['zip_code'] for sub in locations ]
df['zip_codes'] = zip_codes

In [20]:
df.to_csv('brooklyn_queens.csv', index = False)

In [ ]:
staten_island = query_pushshift('staten island')

In [150]:
south_staten_island = query_pushshift('5180 Amboy Rd, Staten Island, NY 10312')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [145]:
north_staten_island = query_pushshift('2241 Victory Blvd, Staten Island, NY 10314')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [155]:
east_central_staten_island = query_pushshift('3155 Amboy Rd, Staten Island, NY 10306')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [156]:
west_central_staten_island = query_pushshift('145 E Service Rd, Staten Island, NY 10314')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [112]:
brooklyn = query_pushshift('brooklyn')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [135]:
south_brooklyn = query_pushshift('5025 18th Ave, Brooklyn, NY 11204')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [136]:
north_brooklyn = query_pushshift('118-11 101st Ave, South Richmond Hill, NY 11419')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [157]:
west_brooklyn = query_pushshift('177 2nd Ave, Brooklyn, NY 11215')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [158]:
east_brooklyn = query_pushshift('9006 Church Ave, Brooklyn, NY 11236')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [137]:
south_queens = query_pushshift('179-22 Union Tpke, Fresh Meadows, NY 11366')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [159]:
west_queens = query_pushshift('22-20 36th Ave, Queens, NY 11106')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [160]:
east_queens = query_pushshift('110-14 Farmers Blvd, Hollis, NY 11412')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [138]:
north_queens = query_pushshift('160-11 Willets Point Blvd, Whitestone, NY 11357')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [113]:
queens = query_pushshift('queens')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [114]:
bronx = query_pushshift('bronx')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [139]:
south_bronx = query_pushshift('860 Melrose Ave FRNT 2, The Bronx, NY 10451')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [140]:
north_bronx = query_pushshift('817 Allerton Ave, The Bronx, NY 10467')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [161]:
west_bronx = query_pushshift('6031 Broadway, The Bronx, NY 10471')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [162]:
east_bronx = query_pushshift('1650 E Gun Hill Rd, The Bronx, NY 10469')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [115]:
manhattan = query_pushshift('manhattan')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [163]:
south_west_manhattan = query_pushshift('600 11th Ave, New York, NY 10036')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [164]:
north_east_manhattan = query_pushshift('1590 2nd Ave, New York, NY 10028')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [141]:
south_manhattan = query_pushshift('463 Broadway, New York, NY 10013')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [142]:
north_manhattan = query_pushshift('143 Malcolm X Blvd, New York, NY 10026')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [170]:
tottenvile = query_pushshift('7254 Amboy Rd, Staten Island, NY 10307')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [171]:
woodrow = query_pushshift('901 Huguenot Ave, Staten Island, NY 10312')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [187]:
bay_terrace = query_pushshift('405 Arthur Kill Rd, Staten Island, NY 10308')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [188]:
brighton_heights = query_pushshift('416 Jersey St, Staten Island, NY 10301')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [172]:
homecrest = query_pushshift('2410 Coney Island Ave, Brooklyn, NY 11223')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [173]:
rockaway = query_pushshift('209 Beach 125th St, Rockaway Park, NY 11694')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [175]:
borough_park = query_pushshift('1301 65th St, Brooklyn, NY 11219')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [176]:
bensonhurst = query_pushshift('2101 86th St, Brooklyn, NY 11214')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [177]:
sheepshead_bay = query_pushshift('3432 Nostrand Ave, Brooklyn, NY 11229')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [178]:
long_island_city = query_pushshift('22-25 Jackson Ave, Long Island City, NY 11101')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [179]:
forest_hills = query_pushshift('73-05 Austin St, Forest Hills, NY 11375')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [180]:
jackson_heights = query_pushshift('92-12 37th Ave, Jackson Heights, NY 11372')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [181]:
woodside = query_pushshift('56-15 Northern Blvd, Woodside, NY 11377')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [182]:
bayside = query_pushshift('205-10 48th Ave, Oakland Gardens, NY 11364')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [183]:
parkchester = query_pushshift('2079 Benedict Ave, The Bronx, NY 10462')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [184]:
williamsbridge = query_pushshift('3357 Fish Ave, The Bronx, NY 10469')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [185]:
riverdale = query_pushshift('3708 Riverdale Ave, The Bronx, NY 10463')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [186]:
belmont = query_pushshift('603 Crescent Ave, The Bronx, NY 10458')

Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp.com/v3/businesses/search
Querying from: https://api.yelp

In [190]:
yelp =pd.concat([yelp1, bronx, brooklyn, queens, staten_island, manhattan,
                north_bronx, south_bronx, north_queens, south_queens, north_staten_island,
                south_staten_island, north_brooklyn, south_brooklyn, north_manhattan, south_manhattan,
                east_central_staten_island, west_central_staten_island, east_brooklyn, west_brooklyn,
                east_bronx, west_bronx, south_west_manhattan, north_east_manhattan, tottenvile, woodrow, 
                bay_terrace, brighton_heights, homecrest, rockaway, borough_park, bensonhurst, sheepshead_bay, 
                long_island_city, forest_hills, jackson_heights, woodside, bayside, parkchester, 
                williamsbridge, riverdale, belmont])

In [191]:
yelp.to_csv('datasets/yelp.csv', index = False)

In [192]:
yelp.shape

(35786, 16)